<div style="background-image:url(images/meschede-seismic-waves.png); padding: 10px 30px 20px 30px; background-size:cover; background-opacity:50%; border-radius:5px">
<p style="float:right; margin-top:20px; padding: 20px 20px 0px 20px; background:rgba(255,255,255,0.6); border-radius:10px;">
<img width="400px" src=images/obspy_logo_full_524x179px.png?raw=true>
</p>

<h1 style="color:#999">ObsPy Workshop</h1>
<h5 style="color:#FFF">Münster, July 7th & 8th</h5>

<h3 style="color:#EEE">Data Acquisition and Processing with ObsPy</h3>

<h2 style="color:#FFF">Handling Event Metadata</h2>
</div>

In [ ]:
%pylab inline
from __future__ import print_function
import matplotlib.pylab as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 12, 8

- for event metadata, the de-facto standard is [QuakeML (an xml document structure)](https://quake.ethz.ch/quakeml/)
- QuakeML files can be read using **`readEvents()`**

In [ ]:
from obspy import readEvents

catalog = readEvents("./data/event_tohoku_with_big_aftershocks.xml")
print(catalog)

- **`readEvents()`** function returns a **`Catalog`** object, which is
a collection of **`Event`** objects.

In [ ]:
print(type(catalog))
print(type(catalog[0]))

In [ ]:
event = catalog[0]
print(event)

- Event objects are again collections of other resources.
- the nested ObsPy Event class structure (Catalog/Event/Origin/Magnitude/FocalMechanism/...) is closely modelled after QuakeML
<img src="images/Event.svg" width=90%>

In [ ]:
print(type(event.origins))
print(type(event.origins[0]))
print(event.origins[0])

In [ ]:
print(type(event.magnitudes))
print(type(event.magnitudes[0]))
print(event.magnitudes[0])

In [ ]:
# try event.<Tab> to get an idea what "children" elements event has

- The Catalog object contains some convenience methods to make
working with events easier.
- for example, the included events can be filtered with various keys.

In [ ]:
largest_magnitude_events = catalog.filter("magnitude >= 7.8")
print(largest_magnitude_events)

- There is a basic preview plot using the matplotlib basemap module.

In [ ]:
catalog.plot(projection="local");

- a (modified) Catalog can be output to file (currently there is write support for QuakeML only)

In [ ]:
largest_magnitude_events.write("/tmp/large_events.xml", format="QUAKEML")
!ls -l /tmp/large_events.xml

- the event type classes can be used to build up Events/Catalogs/Picks/.. from scratch in custom processing work flows and to share them with other researchers in the de facto standard format QuakeML

In [ ]:
from obspy import UTCDateTime
from obspy.core.event import Catalog, Event, Origin, Magnitude
from obspy.core.util.geodetics import FlinnEngdahl

cat = Catalog()
cat.description = "Just a fictitious toy example catalog built from scratch"

e = Event()
e.event_type = "not existing"

o = Origin()
o.time = UTCDateTime(2014, 2, 23, 18, 0, 0)
o.latitude = 47.6
o.longitude = 12.0
o.depth = 10000
o.depth_type = "operator assigned"
o.evaluation_mode = "manual"
o.evaluation_status = "preliminary"
o.region = FlinnEngdahl().get_region(o.longitude, o.latitude)

m = Magnitude()
m.mag = 7.2
m.magnitude_type = "Mw"

m2 = Magnitude()
m2.mag = 7.4
m2.magnitude_type = "Ms"

# also included could be: custom picks, amplitude measurements, station magnitudes,
# focal mechanisms, moment tensors, ...

# make associations, put everything together
cat.append(e)
e.origins = [o]
e.magnitudes = [m, m2]
m.origin_id = o.resource_id
m2.origin_id = o.resource_id

print(cat)
cat.write("/tmp/my_custom_events.xml", format="QUAKEML")
!cat /tmp/my_custom_events.xml

## Acknowledgements

Background picture at the very top is from Matthias Meschede.